In [17]:
import os
import numpy as np
import pandas as pd

def filepaths(sesspath, region, signal):
    region = {'pfc': 'imec0', 'hpc': 'imec1'}[region.lower()]
    signal = {'ap': 'ap', 'lfp': 'lf'}[signal.lower()]
    basedir = sesspath + f'_{region}'
    basename = sesspath.split('/')[-1] + f'_t0.{region}.{signal}'
    datapath = os.path.join(basedir, basename + '.bin')
    metapath = os.path.join(basedir, basename + '.meta')
    return datapath, metapath

def metadata(metapath):
    with open(metapath, 'r') as f:
        md = {}
        for line in f.read().splitlines():
            key, val = line.split('=')
            if key.startswith('~'):
                key = key[1:]
            md[key] = val
    return md

root = '/Users/lukearend/phd/kfx/'
prefix = root + 'data/mnt/Ketamine/'
sessions = pd.read_csv(root + 'ref/sessions.csv')

In [18]:
sess = sessions.iloc[0]
datapath, metapath = filepaths(prefix + sess.path, 'pfc', 'lfp')
md = metadata(metapath)

Imagine session 0 was missing HPC LFP metadata.

Can we reconstruct it using just the PFC LFP and HPC AP metadata?

In [111]:
_, metapath = filepaths(prefix + sessions.iloc[0].path, 'pfc', 'lfp')
pfc_lfp = metadata(metapath)
_, metapath = filepaths(prefix + sessions.iloc[0].path, 'hpc', 'ap')
hpc_ap = metadata(metapath)
_, metapath = filepaths(prefix + sessions.iloc[0].path, 'hpc', 'lfp')
hpc_lfp = metadata(metapath)

In [112]:
assert pfc_lfp['snsSaveChanSubset'] == hpc_lfp['snsSaveChanSubset']
assert pfc_lfp['snsApLfSy'] == hpc_lfp['snsApLfSy']
assert pfc_lfp['imSampRate'] == hpc_lfp['imSampRate']
assert pfc_lfp['snsChanMap'] == hpc_lfp['snsChanMap']

In [113]:
filename = hpc_ap['fileName'].replace('.ap', '.lf')
assert filename == hpc_lfp['fileName']

In [114]:
firstsample = pfc_lfp['firstSample']
assert firstsample == hpc_lfp['firstSample']

In [115]:
# HPC LFP .bin SHA1 returned by 'sha1sum'
sha1 = '48ca6ee8a4f0d6ee1f52445d37034ae60c2208ff'.upper()
assert sha1 == hpc_lfp['fileSHA1']

In [116]:
# HPC LFP .bin size in bytes returned by 'stat'
sizebytes = 13967783830
assert sizebytes == int(hpc_lfp['fileSizeBytes'])

In [117]:
hpc_lfp['fileTimeSecs'], hpc_ap['fileTimeSecs'], pfc_lfp['fileTimeSecs']

('7255.9916', '7255.991266666667', '7256.006')

In [118]:
new = hpc_ap.copy()
new['snsSaveChanSubset'] = pfc_lfp['snsSaveChanSubset']
new['snsApLfSy'] = pfc_lfp['snsApLfSy']
new['imSampRate'] = pfc_lfp['imSampRate']
new['snsChanMap'] = pfc_lfp['snsChanMap']
new['fileName'] = hpc_ap['fileName'].replace('.ap', '.lf')
new['firstSample'] = pfc_lfp['firstSample']
new['fileSHA1'] = '48ca6ee8a4f0d6ee1f52445d37034ae60c2208ff'.upper()
new['fileSizeBytes'] = str(13967783830)
new['fileTimeSecs'] = hpc_ap['fileTimeSecs']
del new['snsShankMap']

In [119]:
for k, actual in new.items():
    expected = hpc_lfp[k]
    if actual != expected:
        print(k, expected, actual)

fileTimeSecs 7255.9916 7255.991266666667


Yes we can, the only discrepancy being error of order 100 microseconds in `fileTimeSecs` copied from the associated HPC AP file.

In [120]:
def reconstruct_hpc_lfp_metadata(pfc_lfp, hpc_ap):
    new = hpc_ap.copy()
    new['snsSaveChanSubset'] = pfc_lfp['snsSaveChanSubset']
    new['snsApLfSy'] = pfc_lfp['snsApLfSy']
    new['imSampRate'] = pfc_lfp['imSampRate']
    new['snsChanMap'] = pfc_lfp['snsChanMap']
    new['fileName'] = hpc_ap['fileName'].replace('.ap', '.lf')
    new['firstSample'] = pfc_lfp['firstSample']
    new['fileSHA1'] = '48ca6ee8a4f0d6ee1f52445d37034ae60c2208ff'.upper()
    new['fileSizeBytes'] = str(13967783830)
    new['fileTimeSecs'] = hpc_ap['fileTimeSecs']
    del new['snsShankMap']
    return new

In [121]:
sess = sessions.iloc[1]
_, metapath = filepaths(sess.path, 'pfc', 'lfp')
pfc_lfp = metadata(prefix + metapath)
_, metapath = filepaths(sess.path, 'hpc', 'ap')
hpc_ap = metadata(prefix + metapath)
new = reconstruct_hpc_lfp_metadata(pfc_lfp, hpc_ap)

In [122]:
_, metapath = filepaths(sess.path, 'hpc', 'lfp')
hpc_lfp = metadata(prefix + metapath)

In [123]:
for k, actual in new.items():
    expected = hpc_lfp[k]
    if actual != expected:
        print(k, expected, actual)

fileSHA1 FB9D77501ACE2E78820A498B1DC3818050383166 48CA6EE8A4F0D6EE1F52445D37034AE60C2208FF
fileSizeBytes 14005579280 13967783830
fileTimeSecs 7275.6256 7275.625533333333


I computed size in bytes and SHA1 for the missing HPC LFP file (session 6). They are listed below.

In [130]:
sessions.iloc[6].path

'2022-08-08-04-05-00_M017_SAL_mPFC_HPC_0_0_0mpk/2022-08-08-04-05-00_M017_SAL_mPFC_HPC_0_0_0mpk_g0'

```
monk 2022-08-08-04-05-00_M017_SAL_mPFC_HPC_0_0_0mpk_g0_imec1 [17:28:20] $ stat 2022-08-08-04-05-00_M017_SAL_mPFC_HPC_0_0_0mpk_g0_t0.imec1.lf.bin 
  File: `2022-08-08-04-05-00_M017_SAL_mPFC_HPC_0_0_0mpk_g0_t0.imec1.lf.bin'
  Size: 13964344240	Blocks: 23429120   IO Block: 131072 regular file
Device: 25h/37d	Inode: 877414      Links: 1
Access: (0750/-rwxr-x---)  Uid: ( 1005/ eunhyep)   Gid: (  100/   users)
Access: 2023-01-30 14:52:49.160192192 -0500
Modify: 2022-08-08 22:15:18.567270833 -0400
Change: 2023-05-03 13:59:07.259301917 -0400
monk 2022-08-08-04-05-00_M017_SAL_mPFC_HPC_0_0_0mpk_g0_imec1 [17:28:30] $ sha1sum 2022-08-08-04-05-00_M017_SAL_mPFC_HPC_0_0_0mpk_g0_t0.imec1.lf.bin 
0a9f9c4585379a4866ae342e7f3c8f5085998e73  2022-08-08-04-05-00_M017_SAL_mPFC_HPC_0_0_0mpk_g0_t0.imec1.lf.bin
```

In [132]:
str(13964344240), '0a9f9c4585379a4866ae342e7f3c8f5085998e73'.upper()

('13964344240', '0A9F9C4585379A4866AE342E7F3C8F5085998E73')